In [33]:
!pip install pypdf
!pip install faiss-cpu
!pip install tiktoken
!pip install qdrant-client
!pip install PyPDF2


[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1.2 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


#### Create vectorDb

In [34]:
!docker-compose up -d

 Container prototypenotebooks-qdrant-1  Running


In [35]:
! curl http://localhost:6333

{"title":"qdrant - vector search engine","version":"1.3.0"}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100    59  100    59    0     0  14575      0 --:--:-- --:--:-- --:--:-- 14750


In [36]:

from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.schema import Document
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import create_citation_fuzzy_match_chain
##############################################################################################################
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback

from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from IPython.display import display, Markdown, Latex


### Same code as the gptLLM

In [ ]:
apiKey = ""

model_name = "Gpt"
llm = ChatOpenAI(openai_api_key=apiKey, model_name="gpt-3.5-turbo-16k")

In [38]:
from langchain.vectorstores import Qdrant
from langchain.embeddings.openai import OpenAIEmbeddings

### There should definitely be a easier way to do this 
#### Here Vector database will be initialized

In [39]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter


loader = PyPDFLoader("data/EStG.pdf")
pages = loader.load_and_split()
print("pages",pages[0])

print("pages",pages[0].metadata["page"])

# Split the text into chunks of 600 characters with 100 characters overlap
text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=600,
        chunk_overlap=100,
        length_function=len
    )
#chunks = text_splitter.split_text(text)
texts = []
for page in pages:
    texts.append([page.metadata["page"]+1, page.page_content])

metadata=[{"page":text[0], "source": text[1]} for text in texts]
embeddings = OpenAIEmbeddings(openai_api_key=apiKey)
completeText=[sublist[1] for sublist in texts if len(sublist) > 1]
# Convert the chunks of text into embeddings to form a knowledge base
vectorDB = Qdrant.from_texts(completeText, embeddings, location=":memory:",metadatas=metadata,collection_name="tax", host="localhost", port=6333)


pages page_content='Ein Service des Bundesministeriums der Justiz sowie des Bundesamts für\nJustiz ‒ www.gesetze-im-internet.de\n- Seite 1 von 280 -Einkommensteuergesetz  (EStG)\nEStG\nAusfertigungsdatum: 16.10.1934\nVollzitat:\n"Einkommensteuergesetz in der Fassung der Bekanntmachung vom 8. Oktober 2009 (BGBl. I S. 3366, 3862), das\nzuletzt durch Artikel 8 Absatz 3 des Gesetzes vom 20. Dezember 2022 (BGBl. I S. 2730) geändert worden ist"\nStand: Neugefasst durch Bek. v. 8.10.2009 I 3366, 3862;\n\xa0 zuletzt geändert durch Art. 8 Abs. 3 G v. 20.12.2022 I 2730\nFußnote\n(+++ Textnachweis Geltung ab: 30.12.1981 +++)\n(+++ Zur Anwendung vgl. § 4, § 4j, § 9, § 10, § 10a, § 13a, 13b, 19a, \n     § 20, § 22, § 32b, § 33, § 37, § 45e, §§ 52 ff. u.\n     § 92a +++)\n(+++ Zur Anwendung vgl. § 19 InvStG +++)\n(+++ Zur Anwendung d. § 34c vgl. § 26 KStG 1977 +++)\n(+++ Zur Anwendung vgl. §§ 6, 16, 19, 30, 31, 34, 38, 41, 42, 43, 45, 47, 49,  \n      50 u. 53 InvStG 2018 +++)\n\xa0\n \nÜberschrift:

### Create the model for retrieval

In [40]:
template = """Use the following pieces of context to answer the question at the end. You can use your previous knowledge to answer the question.
Context: 
{summaries}
"""

messages = [
    SystemMessagePromptTemplate.from_template(template),
    HumanMessagePromptTemplate.from_template("{question}")
]
prompt = ChatPromptTemplate.from_messages(messages)
chain_type_kwargs = {"prompt": prompt}
llm = ChatOpenAI(openai_api_key=apiKey, model_name="gpt-3.5-turbo-16k")

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vectorDB.as_retriever(search_kwargs={"k": 5}),
    return_source_documents=True,
    chain_type_kwargs=chain_type_kwargs
)

### For nice Visualization

In [41]:
backslash_char = "\n " # This is a hack to get around the fact that the backslash is a special character in python strings
def print_result(result):
  output_text = f"""### Question: 
  {query}
  ### Answer: 
  {result['answer']}
  ### Sources: 
  {result['sources']}
  ### All relevant sources:
  {' '.join([f"{backslash_char}"+"PAGE:"+str(doc.metadata['page'])+f"{backslash_char}"+doc.metadata['source']+f"{backslash_char}" for doc in result['source_documents']])}
  """
  return(output_text)
print(' '.join([f"{backslash_char}"+"PAGE:"+str(doc.metadata['page'])+f"{backslash_char}"+doc.metadata['source']+f"{backslash_char}" for doc in result['source_documents']]))

query = "als ein lediger Person, ohne Kinder, mit dem Brutto Gehalt 50k, was ist mein Steuerklasse?"
result = chain(query)
display(Markdown(print_result(result)))


 PAGE:152
 Ein Service des Bundesministeriums der Justiz sowie des Bundesamts für
Justiz ‒ www.gesetze-im-internet.de
- Seite 152 von 280 -(1) 1Für die Durchführung des Lohnsteuerabzugs werden Arbeitnehmer in Steuerklassen eingereiht. 2Dabei gilt
Folgendes:
1.   In die Steuerklasse I gehören Arbeitnehmer, die
a)   unbeschränkt einkommensteuerpflichtig und
aa)   ledig sind,
 
bb)   verheiratet, verwitwet oder geschieden sind und bei denen die Voraussetzungen für die
Steuerklasse III oder IV nicht erfüllt sind; oder
 
 
b)   beschränkt einkommensteuerpflichtig sind;
 
 
2.   in die Steuerklasse II gehören die unter Nummer 1 Buchstabe a bezeichneten Arbeitnehmer, wenn bei
ihnen der Entlastungsbetrag für Alleinerziehende (§ 24b) zu berücksichtigen ist;
 
3.   in die Steuerklasse III gehören Arbeitnehmer,
a)   die verheiratet sind, wenn beide Ehegatten unbeschränkt einkommensteuerpflichtig sind und nicht
dauernd getrennt leben und der Ehegatte des Arbeitnehmers auf Antrag beider Ehegatten 

### Question: 
  als ein lediger Person, ohne Kinder, mit dem Brutto Gehalt 50k, was ist mein Steuerklasse?
  ### Answer: 
  Nach den gegebenen Informationen bist du als ledige Person ohne Kinder in der Steuerklasse I eingereiht.
  ### Sources: 
  
  ### All relevant sources:
  
 PAGE:152
 Ein Service des Bundesministeriums der Justiz sowie des Bundesamts für
Justiz ‒ www.gesetze-im-internet.de
- Seite 152 von 280 -(1) 1Für die Durchführung des Lohnsteuerabzugs werden Arbeitnehmer in Steuerklassen eingereiht. 2Dabei gilt
Folgendes:
1.   In die Steuerklasse I gehören Arbeitnehmer, die
a)   unbeschränkt einkommensteuerpflichtig und
aa)   ledig sind,
 
bb)   verheiratet, verwitwet oder geschieden sind und bei denen die Voraussetzungen für die
Steuerklasse III oder IV nicht erfüllt sind; oder
 
 
b)   beschränkt einkommensteuerpflichtig sind;
 
 
2.   in die Steuerklasse II gehören die unter Nummer 1 Buchstabe a bezeichneten Arbeitnehmer, wenn bei
ihnen der Entlastungsbetrag für Alleinerziehende (§ 24b) zu berücksichtigen ist;
 
3.   in die Steuerklasse III gehören Arbeitnehmer,
a)   die verheiratet sind, wenn beide Ehegatten unbeschränkt einkommensteuerpflichtig sind und nicht
dauernd getrennt leben und der Ehegatte des Arbeitnehmers auf Antrag beider Ehegatten in die
Steuerklasse V eingereiht wird,
 
b)   die verwitwet sind, wenn sie und ihr verstorbener Ehegatte im Zeitpunkt seines Todes
unbeschränkt einkommensteuerpflichtig waren und in diesem Zeitpunkt nicht dauernd getrennt
gelebt haben, für das Kalenderjahr, das dem Kalenderjahr folgt, in dem der Ehegatte verstorben
ist,
 
c)   deren Ehe aufgelöst worden ist, wenn
aa)   im Kalenderjahr der Auflösung der Ehe beide Ehegatten unbeschränkt
einkommensteuerpflichtig waren und nicht dauernd getrennt gelebt haben und
 
bb)   der andere Ehegatte wieder geheiratet hat, von seinem neuen Ehegatten nicht dauernd
getrennt lebt und er und sein neuer Ehegatte unbeschränkt einkommensteuerpflichtig
sind,
 
für das Kalenderjahr, in dem die Ehe aufgelöst worden ist;
 
 
4.   in die Steuerklasse IV gehören Arbeitnehmer, die verheiratet sind, wenn beide Ehegatten unbeschränkt
einkommensteuerpflichtig sind und nicht dauernd getrennt leben; dies gilt auch, wenn einer der
Ehegatten keinen Arbeitslohn bezieht und kein Antrag nach Nummer 3 Buchstabe a gestellt worden ist;
 
5.   in die Steuerklasse V gehören die unter Nummer 4 bezeichneten Arbeitnehmer, wenn der Ehegatte des
Arbeitnehmers auf Antrag beider Ehegatten in die Steuerklasse III eingereiht wird;
 
6.   die Steuerklasse VI gilt bei Arbeitnehmern, die nebeneinander von mehreren Arbeitgebern Arbeitslohn
beziehen, für die Einbehaltung der Lohnsteuer vom Arbeitslohn aus dem zweiten und einem weiteren
Dienstverhältnis sowie in den Fällen des § 39c.
 
3Als unbeschränkt einkommensteuerpflichtig im Sinne der Nummern 3 und 4 gelten nur Personen, die die
Voraussetzungen des § 1 Absatz 1 oder 2 oder des § 1a erfüllen.
(2) 1Für ein minderjähriges und nach § 1 Absatz 1 unbeschränkt einkommensteuerpflichtiges Kind im Sinne des §
32 Absatz 1 Nummer 1 und Absatz 3 werden bei der Anwendung der Steuerklassen I bis IV die Kinderfreibeträge
als Lohnsteuerabzugsmerkmal nach § 39 Absatz 1 wie folgt berücksichtigt:
1.   mit Zähler 0,5, wenn dem Arbeitnehmer der Kinderfreibetrag nach § 32 Absatz 6 Satz 1 zusteht, oder
 
2.   mit Zähler 1, wenn dem Arbeitnehmer der Kinderfreibetrag zusteht, weil
a)   die Voraussetzungen des § 32 Absatz 6 Satz 2 vorliegen oder
 
b)   der andere Elternteil vor dem Beginn des Kalenderjahres verstorben ist oder
 
c)   der Arbeitnehmer allein das Kind angenommen hat.
 
 
2Soweit dem Arbeitnehmer Kinderfreibeträge nach § 32 Absatz 1 bis 6 zustehen, die nicht nach Satz 1
berücksichtigt werden, ist die Zahl der Kinderfreibeträge auf Antrag vorbehaltlich des § 39a Absatz 1 Nummer
6 zu Grunde zu legen. 3In den Fällen des Satzes 2 können die Kinderfreibeträge für mehrere Jahre gelten,
  
 PAGE:163
 Ein Service des Bundesministeriums der Justiz sowie des Bundesamts für
Justiz ‒ www.gesetze-im-internet.de
- Seite 163 von 280 -der Steuerklasse IV für jeden Ehegatten. 5Maßgeblich sind die Steuerbeträge des Kalenderjahres, für das der
Faktor erstmals gelten soll.6In die Bemessungsgrundlage für Y werden jeweils neben den Jahresarbeitslöhnen
der ersten Dienstverhältnisse zusätzlich nur Beträge einbezogen, die nach § 39a Absatz 1 Satz 1 Nummer 1 bis 6
als Freibetrag ermittelt und als Lohnsteuerabzugsmerkmal gebildet werden könnten; Freibeträge werden neben
dem Faktor nicht als Lohnsteuerabzugsmerkmal gebildet. 7In den Fällen des § 39a Absatz 1 Satz 1 Nummer 7
sind bei der Ermittlung von Y und X die Hinzurechnungsbeträge zu berücksichtigen; die Hinzurechnungsbeträge
sind zusätzlich als Lohnsteuerabzugsmerkmal für das erste Dienstverhältnis zu bilden. 8Arbeitslöhne aus zweiten
und weiteren Dienstverhältnissen (Steuerklasse VI) sind im Faktorverfahren nicht zu berücksichtigen. 9Der nach
Satz 1 gebildete Faktor gilt bis zum Ablauf des Kalenderjahres, das auf das Kalenderjahr folgt, in dem der Faktor
erstmals gilt oder zuletzt geändert worden ist. 10Die Ehegatten können eine Änderung des Faktors beantragen,
wenn sich die für die Ermittlung des Faktors maßgeblichen Jahresarbeitslöhne im Sinne des Satzes 6 ändern.
11Besteht eine Anzeigepflicht nach § 39a Absatz 1 Satz 5 oder wird eine Änderung des Freibetrags nach § 39a
Absatz 1 Satz 4 beantragt, gilt die Anzeige oder der Antrag auf Änderung des Freibetrags zugleich als Antrag auf
Anpassung des Faktors.
(2) Für die Einbehaltung der Lohnsteuer vom Arbeitslohn hat der Arbeitgeber Steuerklasse IV und den Faktor
anzuwenden.
(3) 1§ 39 Absatz 6 Satz 3 und 5 gilt mit der Maßgabe, dass die Änderungen nach Absatz 1 Satz 10 und 11 keine
Änderungen im Sinne des § 39 Absatz 6 Satz 3 sind. 2§ 39a ist anzuwenden mit der Maßgabe, dass ein Antrag
nach amtlich vorgeschriebenem Vordruck (§ 39a Absatz 2) nur erforderlich ist, wenn bei der Faktorermittlung
zugleich Beträge nach § 39a Absatz 1 Satz 1 Nummer 1 bis 6 berücksichtigt werden sollen.
(4) Das Faktorverfahren ist im Programmablaufplan für die maschinelle Berechnung der Lohnsteuer (§ 39b Absatz
6) zu berücksichtigen.
Fußnote
(+++ § 39f: Zur Anwendung vgl. § 52 +++)
§ 40 Pauschalierung der Lohnsteuer in besonderen Fällen
(1) 1Das Betriebsstättenfinanzamt (§ 41a Absatz 1 Satz 1 Nummer 1) kann auf Antrag des Arbeitgebers
zulassen, dass die Lohnsteuer mit einem unter Berücksichtigung der Vorschriften des § 38a zu ermittelnden
Pauschsteuersatz erhoben wird, soweit
1.   von dem Arbeitgeber sonstige Bezüge in einer größeren Zahl von Fällen gewährt werden oder
 
2.   in einer größeren Zahl von Fällen Lohnsteuer nachzuerheben ist, weil der Arbeitgeber die Lohnsteuer nicht
vorschriftsmäßig einbehalten hat.
 
2Bei der Ermittlung des Pauschsteuersatzes ist zu berücksichtigen, dass die in Absatz 3 vorgeschriebene
Übernahme der pauschalen Lohnsteuer durch den Arbeitgeber für den Arbeitnehmer eine in Geldeswert
bestehende Einnahme im Sinne des § 8 Absatz 1 darstellt (Nettosteuersatz). 3Die Pauschalierung ist in den Fällen
des Satzes 1 Nummer 1 ausgeschlossen, soweit der Arbeitgeber einem Arbeitnehmer sonstige Bezüge von mehr
als 1 000 Euro im Kalenderjahr gewährt. 4Der Arbeitgeber hat dem Antrag eine Berechnung beizufügen, aus der
sich der durchschnittliche Steuersatz unter Zugrundelegung der durchschnittlichen Jahresarbeitslöhne und der
durchschnittlichen Jahreslohnsteuer in jeder Steuerklasse für diejenigen Arbeitnehmer ergibt, denen die Bezüge
gewährt werden sollen oder gewährt worden sind.
(2) 1Abweichend von Absatz 1 kann der Arbeitgeber die Lohnsteuer mit einem Pauschsteuersatz von 25 Prozent
erheben, soweit er
1.   arbeitstäglich Mahlzeiten im Betrieb an die Arbeitnehmer unentgeltlich oder verbilligt abgibt oder
Barzuschüsse an ein anderes Unternehmen leistet, das arbeitstäglich Mahlzeiten an die Arbeitnehmer
  
 PAGE:120
 Ein Service des Bundesministeriums der Justiz sowie des Bundesamts für
Justiz ‒ www.gesetze-im-internet.de
- Seite 120 von 280 -§ 24b  Entlastungsbetrag für Alleinerziehende
(1) 1Allein stehende Steuerpflichtige können einen Entlastungsbetrag von der Summe der Einkünfte abziehen,
wenn zu ihrem Haushalt mindestens ein Kind gehört, für das ihnen ein Freibetrag nach § 32 Absatz 6 oder
Kindergeld zusteht. 2Die Zugehörigkeit zum Haushalt ist anzunehmen, wenn das Kind in der Wohnung des allein
stehenden Steuerpflichtigen gemeldet ist. 3Ist das Kind bei mehreren Steuerpflichtigen gemeldet, steht der
Entlastungsbetrag nach Satz 1 demjenigen Alleinstehenden zu, der die Voraussetzungen auf Auszahlung des
Kindergeldes nach § 64 Absatz 2 Satz 1 erfüllt oder erfüllen würde in Fällen, in denen nur ein Anspruch auf einen
Freibetrag nach § 32 Absatz 6 besteht. 4Voraussetzung für die Berücksichtigung ist die Identifizierung des Kindes
durch die an dieses Kind vergebene Identifikationsnummer (§ 139b der Abgabenordnung). 5Ist das Kind nicht nach
einem Steuergesetz steuerpflichtig (§ 139a Absatz 2 der Abgabenordnung), ist es in anderer geeigneter Weise
zu identifizieren. 6Die nachträgliche Vergabe der Identifikationsnummer wirkt auf Monate zurück, in denen die
Voraussetzungen der Sätze 1 bis 3 vorliegen.
(2) 1Gehört zum Haushalt des allein stehenden Steuerpflichtigen ein Kind im Sinne des Absatzes 1, beträgt der
Entlastungsbetrag im Kalenderjahr 4 260 Euro. 2Für jedes weitere Kind im Sinne des Absatzes 1 erhöht sich der
Betrag nach Satz 1 um 240 Euro je weiterem Kind.
(3) 1Allein stehend im Sinne des Absatzes 1 sind Steuerpflichtige, die nicht die Voraussetzungen
für die Anwendung des Splitting-Verfahrens (§ 26 Absatz 1) erfüllen oder verwitwet sind und keine
Haushaltsgemeinschaft mit einer anderen volljährigen Person bilden, es sei denn, für diese steht ihnen ein
Freibetrag nach § 32 Absatz 6 oder Kindergeld zu oder es handelt sich um ein Kind im Sinne des § 63 Absatz
1 Satz 1, das einen Dienst nach § 32 Absatz 5 Satz 1 Nummer 1 und 2 leistet oder eine Tätigkeit nach § 32
Absatz 5 Satz 1 Nummer 3 ausübt. 2Ist die andere Person mit Haupt- oder Nebenwohnsitz in der Wohnung
des Steuerpflichtigen gemeldet, wird vermutet, dass sie mit dem Steuerpflichtigen gemeinsam wirtschaftet
(Haushaltsgemeinschaft). 3Diese Vermutung ist widerlegbar, es sei denn, der Steuerpflichtige und die andere
Person leben in einer eheähnlichen oder lebenspartnerschaftsähnlichen Gemeinschaft.
(4) Für jeden vollen Kalendermonat, in dem die Voraussetzungen des Absatzes 1 nicht vorgelegen haben,
ermäßigt sich der Entlastungsbetrag nach Absatz 2 um ein Zwölftel.
III.
Veranlagung
§ 25 Veranlagungszeitraum, Steuererklärungspflicht
(1) Die Einkommensteuer wird nach Ablauf des Kalenderjahres (Veranlagungszeitraum) nach dem Einkommen
veranlagt, das der Steuerpflichtige in diesem Veranlagungszeitraum bezogen hat, soweit nicht nach § 43 Absatz 5
und § 46 eine Veranlagung unterbleibt.
(2) (weggefallen)
(3) 1Die steuerpflichtige Person hat für den Veranlagungszeitraum eine eigenhändig unterschriebene
Einkommensteuererklärung abzugeben. 2Wählen Ehegatten die Zusammenveranlagung (§ 26b), haben sie eine
gemeinsame Steuererklärung abzugeben, die von beiden eigenhändig zu unterschreiben ist.
(4) 1Die Erklärung nach Absatz 3 ist nach amtlich vorgeschriebenem Datensatz durch Datenfernübertragung zu
übermitteln, wenn Einkünfte nach § 2 Absatz 1 Satz 1 Nummer 1 bis 3 erzielt werden und es sich nicht um einen
der Veranlagungsfälle gemäß § 46 Absatz 2 Nummer 2 bis 8 handelt. 2Auf Antrag kann die Finanzbehörde zur
Vermeidung unbilliger Härten auf eine Übermittlung durch Datenfernübertragung verzichten.
Fußnote
(+++ § 25: Zur Anwendung vgl. § 52 Abs. 39 u. 68 +++)
(+++ § 25: Zur Anwendung vgl. § 52a Abs. 13 (§ 52a aufgeh. durch Art. 2 Nr. 35 G v. 25.7.2014 I 1266 mWv
31.7.2014) +++)
  
 PAGE:153
 Ein Service des Bundesministeriums der Justiz sowie des Bundesamts für
Justiz ‒ www.gesetze-im-internet.de
- Seite 153 von 280 -wenn nach den tatsächlichen Verhältnissen zu erwarten ist, dass die Voraussetzungen bestehen bleiben. 4Bei
Anwendung der Steuerklassen III und IV sind auch Kinder des Ehegatten bei der Zahl der Kinderfreibeträge zu
berücksichtigen. 5Der Antrag kann nur nach amtlich vorgeschriebenem Vordruck gestellt werden.
(3) 1Auf Antrag des Arbeitnehmers kann abweichend von Absatz 1 oder 2 eine für ihn ungünstigere Steuerklasse
oder geringere Zahl der Kinderfreibeträge als Lohnsteuerabzugsmerkmal gebildet werden. 2Der Wechsel von der
Steuerklasse III oder V in die Steuerklasse IV ist auch auf Antrag nur eines Ehegatten möglich mit der Folge, dass
beide Ehegatten in die Steuerklasse IV eingereiht werden. 3Diese Anträge sind nach amtlich vorgeschriebenem
Vordruck zu stellen und vom Antragsteller eigenhändig zu unterschreiben.
§ 39 Lohnsteuerabzugsmerkmale
(1) 1Für die Durchführung des Lohnsteuerabzugs werden auf Veranlassung des Arbeitnehmers
Lohnsteuerabzugsmerkmale gebildet (§ 39a Absatz 1 und 4, § 39e Absatz 1 in Verbindung mit § 39e Absatz
4 Satz 1 und nach § 39e Absatz 8). 2Soweit Lohnsteuerabzugsmerkmale nicht nach § 39e Absatz 1 Satz 1
automatisiert gebildet werden oder davon abweichend zu bilden sind, ist das Finanzamt für die Bildung der
Lohnsteuerabzugsmerkmale nach den §§ 38b und 39a und die Bestimmung ihrer Geltungsdauer zuständig.
3Für die Bildung der Lohnsteuerabzugsmerkmale sind die von den Meldebehörden nach § 39e Absatz 2 Satz 2
mitgeteilten Daten vorbehaltlich einer nach Satz 2 abweichenden Bildung durch das Finanzamt bindend. 4Die
Bildung der Lohnsteuerabzugsmerkmale ist eine gesonderte Feststellung von Besteuerungsgrundlagen im Sinne
des § 179 Absatz 1 der Abgabenordnung, die unter dem Vorbehalt der Nachprüfung steht. 5Die Bildung und die
Änderung der Lohnsteuerabzugsmerkmale sind dem Arbeitnehmer bekannt zu geben. 6Die Bekanntgabe richtet
sich nach § 119 Absatz 2 der Abgabenordnung und § 39e Absatz 6. 7Der Bekanntgabe braucht keine Belehrung
über den zulässigen Rechtsbehelf beigefügt zu werden. 8Ein schriftlicher Bescheid mit einer Belehrung über den
zulässigen Rechtsbehelf ist jedoch zu erteilen, wenn einem Antrag des Arbeitnehmers auf Bildung oder Änderung
der Lohnsteuerabzugsmerkmale nicht oder nicht in vollem Umfang entsprochen wird oder der Arbeitnehmer die
Erteilung eines Bescheids beantragt. 9Vorbehaltlich des Absatzes 5 ist § 153 Absatz 2 der Abgabenordnung nicht
anzuwenden.
(2) 1Für die Bildung und die Änderung der Lohnsteuerabzugsmerkmale nach Absatz 1 Satz 2 des nach § 1
Absatz 1 unbeschränkt einkommensteuerpflichtigen Arbeitnehmers ist das Wohnsitzfinanzamt im Sinne
des § 19 Absatz 1 Satz 1 und 2 der Abgabenordnung und in den Fällen des Absatzes 4 Nummer 5 das
Betriebsstättenfinanzamt nach § 41a Absatz 1 Satz 1 Nummer 1 zuständig. 2Ist der Arbeitnehmer nach § 1
Absatz 2 unbeschränkt einkommensteuerpflichtig, nach § 1 Absatz 3 als unbeschränkt einkommensteuerpflichtig
zu behandeln oder beschränkt einkommensteuerpflichtig, ist das Betriebsstättenfinanzamt für die Bildung
und die Änderung der Lohnsteuerabzugsmerkmale zuständig. 3Ist der nach § 1 Absatz 3 als unbeschränkt
einkommensteuerpflichtig zu behandelnde Arbeitnehmer gleichzeitig bei mehreren inländischen Arbeitgebern
tätig, ist für die Bildung der weiteren Lohnsteuerabzugsmerkmale das Betriebsstättenfinanzamt zuständig,
das erstmals Lohnsteuerabzugsmerkmale gebildet hat. 4Bei Ehegatten, die beide Arbeitslohn von inländischen
Arbeitgebern beziehen, ist das Betriebsstättenfinanzamt des älteren Ehegatten zuständig.
(3) 1In den Fällen des Absatzes 2 Satz 1 hat der Arbeitnehmer den Antrag für die erstmalige Zu-teilung einer
Identifikationsnummer (§ 139b der Abgabenordnung) beim Wohnsitzfinanzamt und in den Fällen des Absatzes
2 Satz 2 beim Be-triebsstättenfinanzamt zu stellen. 2Die Zuteilung einer Identifikationsnummer kann auch
  
 PAGE:151
 mit der Lohnsteuer für den laufenden Arbeitslohn des Kalenderjahres und für etwa im Kalenderjahr bereits
gezahlte sonstige Bezüge die voraussichtliche Jahreslohnsteuer ergibt.
(4) Bei der Ermittlung der Lohnsteuer werden die Besteuerungsgrundlagen des Einzelfalls durch die Einreihung
der Arbeitnehmer in Steuerklassen (§ 38b), Feststellung von Freibeträgen und Hinzurechnungsbeträgen
(§ 39a) sowie Bereitstellung von elektronischen Lohnsteuerabzugsmerkmalen (§ 39e) oder Ausstellung
von entsprechenden Bescheinigungen für den Lohnsteuerabzug (§ 39 Absatz 3 und § 39e Absatz 7 und 8)
berücksichtigt.
§ 38b  Lohnsteuerklassen, Zahl der Kinderfreibeträge
 
  

In [42]:
while True:
    query = input("Ask a question: ")
    result = chain(query)
    display(Markdown(print_result(result)))

KeyboardInterrupt: 